In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math


def direct_estimator():
    c_list = [i / 100 for i in range(14, 3, -1)]
    max_attemps = 1000000
    relative_error_percent = 5
    x_axis = []
    y_axis = []
    M_y_axis = []
    pc_list = []
    for c in c_list:
        calculated_p_c = 0.5 * (1 - math.erf(1 / math.sqrt(2 * c)))
        # Lets take relative error 5%
        low_limit = (1 - relative_error_percent / 100) * calculated_p_c
        high_limit = (1 + relative_error_percent / 100) * calculated_p_c
        greater_than_1 = 0
        for m in range(1, max_attemps):
            experimental_p_c = greater_than_1 / m
            if low_limit <= experimental_p_c <= high_limit:
                pc_list.append(calculated_p_c)
                break
            w = 0
            for n in range(1, 500):
                xi = np.random.normal(0, 1)
                z = (n - 0.5) * np.pi
                w += xi * np.sin(z * c) / z
            w *= math.sqrt(2)
            if w > 1:
                greater_than_1 += 1
        if m < max_attemps - 1:
            print(f'It took {m} attempts for {c} to reach the relative error of 5%')
            x_axis.insert(0, c)
            y_axis.insert(0, m)
            # =============================================================================================
            # Lets estimate the theoretical number of trials to reach the same relative error
            # For 5% we should take 2 deviations. So absolute_error is approximately 2 * sqrt(pc / n),
            # and relative_error = 0.05 = (pc - absolute_error) / pc = 1 - absolute_error / pc = 1 - 2 * sqrt(1 / (pc * n))
            #  Then n = 4 / (pc * (1 - 0.05)**2)
            M_y_axis.insert(0, 4 / (calculated_p_c * 0.95 ** 2 ))
            # =============================================================================================
        else:
            print(f'For {c} no successful events happened in {max_attemps} attempts')
            break
    plt.plot(x_axis, y_axis, label='empirical M')
    plt.plot(x_axis, M_y_axis, label='theoretical M')
    plt.title('Number of attempts to reach 5% relative error')
    plt.xlabel('c')
    plt.ylabel('Number of attempts')
    plt.legend()
    plt.show()  

In [ ]:
direct_estimator()

In [ ]:
# We see that the number of required attempts grows exponentially as c becomes smaller. 
# For c=0.04 we couldn't even estimate the probability, as we didn't have a single successful event in 1000000 attempts
# That makes sense since the probability for c=0.04 is 0.5*(1-erf(1/sqrt(2*c))) = 2.87*10^(-7)

In [ ]:
# For the importance sampling, we use the exponential distribution lambda * e^(-lambda*(x-1)).
# We select lambda = 1 / sqrt(2*pi*c) so that at x = 1 both exponential and original distributions 
# will have the same value. To implement the (x - 1) part, we use the regular exponential distribution
# from np.random, and then add 1 to the calculated sample. Note, since our sample is >= 1, the random variable is always 1

def original_density(c,x):
    return 1 / math.sqrt(2*np.pi*c) * np.exp(-x**2 / (2*c))

def optimal_importance_sampling():
    c_list = [i / 100 for i in range(14, 1, -1)]
    max_attemps = 1000000
    x_axis = []
    y_axis = []
    for c in c_list:
        def exponential_distribution(x):
            rate = 1 / scale
            return rate * np.e ** (-rate * x)
        scale = math.sqrt(2*np.pi*c)
        calculated_p_c = 0.5 * (1 - math.erf(1 / math.sqrt(2 * c)))
        # Lets take relative error 5%
        low_limit = 0.95 * calculated_p_c
        high_limit = 1.05 * calculated_p_c
        greater_than_1 = 0
        experimental_p_c = 0
        for m in range(1, max_attemps):
            sample = np.random.exponential(scale)
            experimental_p_c += original_density(c, sample + 1) / exponential_distribution(sample)
            if low_limit <= experimental_p_c / m <= high_limit:
                experimental_p_c /= m
                break
        if m < max_attemps - 1:
            print(f'It took {m} attempts for {c} to reach the relative error of 5%')
            x_axis.insert(0, c)
            y_axis.insert(0, m)

        else:
            print(f'For {c} no successful events happened in {max_attemps} attempts')
    plt.plot(x_axis, y_axis)  
    plt.title('Number of attempts to reach 5% relative error')
    plt.xlabel('c')
    plt.ylabel('Number of attempts')
    plt.show()

In [ ]:
optimal_importance_sampling()

In [ ]:
# Using importance sampling, we reduced the number of attempts several orders of magnitude. 
# We were also able to estimate the probability for the really small c